# Setting

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

import torch
from datasets import Dataset, DatasetDict
from transformers import AdamW, EarlyStoppingCallback
from transformers import get_linear_schedule_with_warmup
from transformers import TrainingArguments, Trainer
from transformers import BertForSequenceClassification
from kobert_tokenizer import KoBERTTokenizer


import wandb
import warnings
warnings.filterwarnings('ignore')

/home/k-cat/anaconda3/envs/lth/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
wandb.init(project='grooming', name='0209_run1')

wandb: Currently logged in as: oiehhun (oiehhun-yonsei-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


# Data Load

In [3]:
data = pd.read_csv('/home/k-cat/users/lth/data/GroomingDataset.csv')
data

,source,label,message_idx,author,text
0,PJ chats,1,0,decoy,"안녕, 나 매즈야"
1,PJ chats,1,0,Billy Joe,"안녕, 자기야"
2,PJ chats,1,0,Billy Joe,거의 일 다 끝났어. 정말 기뻐
3,PJ chats,1,0,decoy,안녕 :) 그거 좋은 거지?
4,PJ chats,1,0,Billy Joe,응
...,...,...,...,...,...
24248,nateon,0,26_20,2,역시 좋은 애니에 좋은 음악까지...
24249,nateon,0,26_20,2,중요한 역할을 하는 거 같아!
24250,nateon,0,26_20,1,맞아! 하울의 움직이는 성도 너무 좋았어!
24251,nateon,0,26_20,2,헉 나 진짜 좋아하는 애니야.


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24253 entries, 0 to 24252
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   source       24253 non-null  object
 1   label        24253 non-null  int64 
 2   message_idx  24253 non-null  object
 3   author       24253 non-null  object
 4   text         24253 non-null  object
dtypes: int64(1), object(4)
memory usage: 947.5+ KB


# Data preprocessing

In [5]:
# 텍스트 대체
data['text'] = data['text'].str.replace('키키', 'ㅋㅋ')
data

,source,label,message_idx,author,text
0,PJ chats,1,0,decoy,"안녕, 나 매즈야"
1,PJ chats,1,0,Billy Joe,"안녕, 자기야"
2,PJ chats,1,0,Billy Joe,거의 일 다 끝났어. 정말 기뻐
3,PJ chats,1,0,decoy,안녕 :) 그거 좋은 거지?
4,PJ chats,1,0,Billy Joe,응
...,...,...,...,...,...
24248,nateon,0,26_20,2,역시 좋은 애니에 좋은 음악까지...
24249,nateon,0,26_20,2,중요한 역할을 하는 거 같아!
24250,nateon,0,26_20,1,맞아! 하울의 움직이는 성도 너무 좋았어!
24251,nateon,0,26_20,2,헉 나 진짜 좋아하는 애니야.


In [6]:
# 중복 텍스트 삭제
data = data.drop_duplicates(['text'], keep='first').reset_index(drop=True)
data

,source,label,message_idx,author,text
0,PJ chats,1,0,decoy,"안녕, 나 매즈야"
1,PJ chats,1,0,Billy Joe,"안녕, 자기야"
2,PJ chats,1,0,Billy Joe,거의 일 다 끝났어. 정말 기뻐
3,PJ chats,1,0,decoy,안녕 :) 그거 좋은 거지?
4,PJ chats,1,0,Billy Joe,응
...,...,...,...,...,...
21427,nateon,0,26_20,2,역시 좋은 애니에 좋은 음악까지...
21428,nateon,0,26_20,2,중요한 역할을 하는 거 같아!
21429,nateon,0,26_20,1,맞아! 하울의 움직이는 성도 너무 좋았어!
21430,nateon,0,26_20,2,헉 나 진짜 좋아하는 애니야.


In [7]:
chat_num_dict = {}

for k, v in data['message_idx'].value_counts().items():
    chat_num_dict[k] = v

data['chat_num'] = data['message_idx'].map(chat_num_dict)
data

,source,label,message_idx,author,text,chat_num
0,PJ chats,1,0,decoy,"안녕, 나 매즈야",113
1,PJ chats,1,0,Billy Joe,"안녕, 자기야",113
2,PJ chats,1,0,Billy Joe,거의 일 다 끝났어. 정말 기뻐,113
3,PJ chats,1,0,decoy,안녕 :) 그거 좋은 거지?,113
4,PJ chats,1,0,Billy Joe,응,113
...,...,...,...,...,...,...
21427,nateon,0,26_20,2,역시 좋은 애니에 좋은 음악까지...,32
21428,nateon,0,26_20,2,중요한 역할을 하는 거 같아!,32
21429,nateon,0,26_20,1,맞아! 하울의 움직이는 성도 너무 좋았어!,32
21430,nateon,0,26_20,2,헉 나 진짜 좋아하는 애니야.,32


In [8]:
# source가 kakao, facebook, instagram, nateon인 데이터만 추출
kakao = data[data['source'] == 'kakao']
facebook = data[data['source'] == 'facebook']
instagram = data[data['source'] == 'instagram']
nateon = data[data['source'] == 'nateon']

# 각 source별로 가장 긴 채팅
print(kakao['message_idx'].value_counts().max())
print(facebook['message_idx'].value_counts().max())
print(instagram['message_idx'].value_counts().max())
print(nateon['message_idx'].value_counts().max())


24
29
27
44


In [9]:
grooming = data[:7574]
non_grooming = data[7574:]
grooming

,source,label,message_idx,author,text,chat_num
0,PJ chats,1,0,decoy,"안녕, 나 매즈야",113
1,PJ chats,1,0,Billy Joe,"안녕, 자기야",113
2,PJ chats,1,0,Billy Joe,거의 일 다 끝났어. 정말 기뻐,113
3,PJ chats,1,0,decoy,안녕 :) 그거 좋은 거지?,113
4,PJ chats,1,0,Billy Joe,응,113
...,...,...,...,...,...,...
7569,PJ chats,1,621,decoy,"나 곧 가야 해, 엄마가 소리치고 있어.",31
7570,PJ chats,1,621,jackjohnsons7,그거 좋았어?,31
7571,PJ chats,1,621,jackjohnsons7,그래서 너 하트포드 어디 거리에서 살아?,31
7572,PJ chats,1,621,decoy,너한테 안 알려줘.,31


In [10]:
# message_idx 별로 author 리스트로 저장 (딕셔너리)
author_dict = {}
for idx in grooming['message_idx'].unique():
    author_dict[idx] = set(grooming[grooming['message_idx'] == idx]['author'].tolist())

# author_dict의 value에 decoy가 포함되어 있으면 제거
for idx, authors in author_dict.items():
    if 'decoy' in authors:
        author_dict[idx].remove('decoy')

In [11]:
author_dict = {'0': {'Billy Joe'},
 '1': {'Billy Joe'},
 '2': {'Billy Joe'},
 '3': {'Billy Joe'},
 '4': {'Billy Joe'},
 '5': {'Billy Joe'},
 '6': {'Billy Joe'},
 '7': {'Billy Joe'},
 '8': {'Carlos Cabrera'},
 '9': {'Carlos Cabrera'},
 '10': {'Carlos Cabrera'},
 '11': {'Carlos Cabrera'},
 '12': {'Carlos Cabrera'},
 '13': {'Carlos Cabrera'},
 '14': {'Carlos Cabrera'},
 '15': {'Carlos Cabrera'},
 '16': {'Carlos Cabrera'},
 '17': {'Carlos Cabrera'},
 '18': {'Carlos Cabrera'},
 '19': {'DB'},
 '20': {'DB'},
 '21': {'DB'},
 '22': {'DB'},
 '23': {'DB'},
 '24': {'DB'},
 '25': {'DB'},
 '26': {'DB'},
 '27': {'DB'},
 '28': {'DB'},
 '29': {'DB'},
 '30': {'DB'},
 '31': {'DB'},
 '32': {'DB'},
 '33': {'DB'},
 '34': {'DB'},
 '35': {'DB'},
 '36': {'DB'},
 '37': {'DB'},
 '38': {'DB'},
 '39': {'DB'},
 '40': {'DB'},
 '41': {'DB'},
 '42': {'DB'},
 '43': {'DB'},
 '44': {'DB'},
 '45': {'DB'},
 '46': {'DB'},
 '47': {'DB'},
 '48': {'DB'},
 '49': {'DB'},
 '50': {'DB'},
 '51': {'DB'},
 '52': {'DB'},
 '53': {'DB'},
 '54': {'DB'},
 '55': {'DB'},
 '57': {'DB'},
 '58': {'DB'},
 '59': {'DB'},
 '60': {'DB'},
 '61': {'DB'},
 '62': {'DB'},
 '63': {'DB'},
 '64': {'DB'},
 '65': {'DB'},
 '66': {'DB'},
 '67': {'DB'},
 '68': {'DB'},
 '69': {'DB'},
 '70': {'DB'},
 '71': {'DB'},
 '72': {'DB'},
 '73': {'DB'},
 '74': {'DB'},
 '75': {'DB'},
 '76': {'DB'},
 '77': {'DB'},
 '78': {'DB'},
 '79': {'DB'},
 '80': {'DB'},
 '81': {'DB'},
 '82': {'DB'},
 '83': {'DB'},
 '85': {'DB'},
 '86': {'DB'},
 '87': {'DB'},
 '88': {'DB'},
 '89': {'DB'},
 '90': {'DB'},
 '91': {'DB'},
 '93': {'DB'},
 '94': {'DB'},
 '95': {'DB'},
 '96': {'DB'},
 '97': {'DB'},
 '98': {'DB'},
 '99': {'DB'},
 '100': {'DB'},
 '101': {'DB'},
 '102': {'DB'},
 '103': {'DB'},
 '104': {'DB'},
 '105': {'DB'},
 '106': {'DB'},
 '107': {'DB'},
 '108': {'DB'},
 '109': {'DB'},
 '110': {'DB'},
 '113': {'DB'},
 '114': {'DB'},
 '115': {'DB'},
 '116': {'DB'},
 '117': {'DB'},
 '118': {'DB'},
 '119': {'DB'},
 '120': {'DB'},
 '121': {'DB'},
 '122': {'DB'},
 '123': {'DB'},
 '124': {'DB'},
 '125': {'DB'},
 '126': {'DB'},
 '127': {'DB'},
 '128': {'DB'},
 '129': {'DB'},
 '130': {'DB'},
 '131': {'DB'},
 '132': {'DB'},
 '133': {'DB'},
 '134': {'DB'},
 '135': {'DB'},
 '136': {'DB'},
 '137': {'DB'},
 '138': {'DB'},
 '139': {'DB'},
 '140': {'DB'},
 '141': {'DB'},
 '144': {'DB'},
 '145': {'DB'},
 '146': {'DB'},
 '147': {'DB'},
 '148': {'DB'},
 '149': {'DB'},
 '150': {'DB'},
 '152': {'DB'},
 '153': {'DB'},
 '154': {'DB'},
 '155': {'DB'},
 '156': {'DB'},
 '157': {'DB'},
 '158': {'DB'},
 '160': {'DB'},
 '161': {'DB'},
 '162': {'DB'},
 '163': {'DB'},
 '164': {'DB'},
 '165': {'DB'},
 '166': {'DB'},
 '167': {'F C'},
 '168': {'F C'},
 '169': {'F C'},
 '170': {'F C'},
 '171': {'F C'},
 '172': {'F C'},
 '173': {'F C'},
 '174': {'F C'},
 '175': {'F C'},
 '176': {'F C'},
 '177': {'F C'},
 '178': {'F C'},
 '179': {'F C'},
 '180': {'F C'},
 '181': {'F C'},
 '182': {'F C'},
 '183': {'F C'},
 '184': {'F C'},
 '185': {'F C'},
 '186': {'F C'},
 '187': {'F C'},
 '188': {'F C'},
 '189': {'F C'},
 '190': {'F C'},
 '191': {'F C'},
 '192': {'F C'},
 '193': {'J H'},
 '194': {'J H'},
 '195': {'J H'},
 '196': {'J H'},
 '197': {'J H'},
 '198': {'Josh Mcclendon'},
 '199': {'Josh Mcclendon'},
 '200': {'Josh Mcclendon'},
 '201': {'Josh Mcclendon'},
 '202': {'Josh Mcclendon'},
 '203': {'Josh Mcclendon'},
 '205': {'Josh Mcclendon'},
 '206': {'Josh Mcclendon'},
 '207': {'Josh Mcclendon'},
 '209': {'Josh Mcclendon'},
 '210': {'Josh Mcclendon'},
 '211': {'Josh Mcclendon'},
 '212': {'Josh Mcclendon'},
 '213': {'Josh Mcclendon'},
 '214': {'Josh Mcclendon'},
 '215': {'Josh Mcclendon'},
 '216': {'Josh Mcclendon'},
 '217': {'Josh Mcclendon'},
 '218': {'Josh Mcclendon'},
 '219': {'Josh Mcclendon'},
 '220': {'Josh Mcclendon'},
 '221': {'Josh Mcclendon'},
 '222': {'Josh Mcclendon'},
 '223': {'Josh Mcclendon'},
 '224': {'Josh Mcclendon'},
 '225': {'Josh Mcclendon'},
 '226': {'Josh Mcclendon'},
 '227': {'Josh Mcclendon'},
 '228': {'Josh Mcclendon'},
 '229': {'Josh Mcclendon'},
 '230': {'Josh Mcclendon'},
 '231': {'Josh Mcclendon'},
 '232': {'Josh Mcclendon'},
 '233': {'Josh Mcclendon'},
 '235': {'Josh Mcclendon'},
 '236': {'Josh Mcclendon'},
 '237': {'Josh Mcclendon'},
 '238': {'Josh Mcclendon'},
 '239': {'Josh Mcclendon'},
 '240': {'Josh Mcclendon'},
 '241': {'Josh Mcclendon'},
 '242': {'Josh Mcclendon'},
 '243': {'Josh Mcclendon'},
 '244': {'Josh Mcclendon'},
 '245': {'Josh Mcclendon'},
 '246': {'Josh Mcclendon'},
 '247': {'Josh Mcclendon'},
 '248': {'Josh Mcclendon'},
 '249': {'Josh Mcclendon'},
 '250': {'Josh Mcclendon'},
 '251': {'Josh Mcclendon'},
 '252': {'Josh Mcclendon'},
 '253': {'Josh Mcclendon'},
 '254': {'Josh Mcclendon'},
 '255': {'Josh Mcclendon'},
 '256': {'Josh Mcclendon'},
 '257': {'Josh Mcclendon'},
 '258': {'Josh Mcclendon'},
 '259': {'Josh Mcclendon'},
 '260': {'Josh Mcclendon'},
 '261': {'Josh Mcclendon'},
 '262': {'Josh Mcclendon'},
 '263': {'Josh Mcclendon'},
 '264': {'Josh Mcclendon'},
 '265': {'Josh Mcclendon'},
 '266': {'Josh Mcclendon'},
 '267': {'Josh Mcclendon'},
 '268': {'Josh Mcclendon'},
 '269': {'Josh Mcclendon'},
 '270': {'Josh Mcclendon'},
 '271': {'Josh Mcclendon'},
 '272': {'Josh Mcclendon'},
 '273': {'Josh Mcclendon'},
 '274': {'Josh Mcclendon'},
 '275': {'Josh Mcclendon'},
 '276': {'Josh Mcclendon'},
 '277': {'Josh Mcclendon'},
 '278': {'Josh Mcclendon'},
 '279': {'Josh Mcclendon'},
 '280': {'Josh Mcclendon'},
 '281': {'Josh Mcclendon'},
 '282': {'Josh Mcclendon'},
 '284': {'Josh Mcclendon'},
 '285': {'Roger Cassidy'},
 '286': {'Roger Cassidy'},
 '287': {'Ryan Thompson'},
 '288': {'Ryan Thompson'},
 '289': {'Ryan Thompson'},
 '290': {'Ryan Thompson'},
 '291': {'Ryan Thompson'},
 '292': {'Ryan Thompson'},
 '293': {'Ryan Thompson'},
 '294': {'Ryan Thompson'},
 '295': {'Ryan Thompson'},
 '296': {'Ryan Thompson'},
 '297': {'Ryan Thompson'},
 '298': {'Ryan Thompson'},
 '299': {'Ryan Thompson'},
 '300': {'Ryan Thompson'},
 '301': {'Ryan Thompson'},
 '302': {'Ryan Thompson'},
 '303': {'Ryan Thompson'},
 '304': {'Ryan Thompson'},
 '305': {'allenriley2011'},
 '306': {'allenriley2011'},
 '308': {'allenriley2011'},
 '310': {'allenriley2011'},
 '311': {'allenriley2011'},
 '312': {'allenriley2011'},
 '313': {'allenriley2011'},
 '314': {'allenriley2011'},
 '315': {'allenriley2011'},
 '316': {'allenriley2011'},
 '317': {'allenriley2011'},
 '318': {'allenriley2011'},
 '319': {'allenriley2011'},
 '320': {'allenriley2011'},
 '321': {'allenriley2011'},
 '323': {'allenriley2011'},
 '324': {'allenriley2011'},
 '325': {'allenriley2011'},
 '326': {'allenriley2011'},
 '327': {'allenriley2011'},
 '328': {'allenriley2011'},
 '329': {'allenriley2011'},
 '330': {'allenriley2011'},
 '331': {'allenriley2011'},
 '332': {'allenriley2011'},
 '333': {'allenriley2011'},
 '334': {'allenriley2011'},
 '335': {'allenriley2011'},
 '336': {'allenriley2011'},
 '338': {'allenriley2011'},
 '339': {'allenriley2011'},
 '340': {'allenriley2011'},
 '341': {'allenriley2011'},
 '342': {'allenriley2011'},
 '343': {'allenriley2011'},
 '344': {'allenriley2011'},
 '345': {'allenriley2011'},
 '346': {'allenriley2011'},
 '347': {'allenriley2011'},
 '348': {'allenriley2011'},
 '349': {'allenriley2011'},
 '351': {'allenriley2011'},
 '352': {'allenriley2011'},
 '353': {'allenriley2011'},
 '354': {'allenriley2011'},
 '355': {'allenriley2011'},
 '356': {'allenriley2011'},
 '357': {'allenriley2011'},
 '358': {'allenriley2011'},
 '359': {'allenriley2011'},
 '360': {'bossofct'},
 '361': {'bossofct'},
 '365': {'chrisnr1998'},
 '366': {'chrisnr1998'},
 '367': {'chrisnr1998'},
 '368': {'chrisnr1998'},
 '371': {'chrisnr1998'},
 '372': {'chrisnr1998'},
 '373': {'chrisnr1998'},
 '374': {'chrisnr1998'},
 '375': {'chrisnr1998'},
 '377': {'chrisnr1998'},
 '379': {'chrisnr1998'},
 '380': {'chrisnr1998'},
 '381': {'chrisnr1998'},
 '382': {'chrisnr1998'},
 '383': {'chrisnr1998'},
 '384': {'chrisnr1998'},
 '385': {'chrisnr1998'},
 '388': {'chrisnr1998'},
 '389': {'chrisnr1998'},
 '390': {'chrisnr1998'},
 '391': {'chrisnr1998'},
 '392': {'chrisnr1998'},
 '395': {'chrisnr1998'},
 '396': {'chrisnr1998'},
 '397': {'chrisnr1998'},
 '398': {'chrisnr1998'},
 '399': {'chrisnr1998'},
 '400': {'chrisnr1998'},
 '401': {'chrisnr1998'},
 '402': {'chrisnr1998'},
 '403': {'chrisnr1998'},
 '404': {'chrisnr1998'},
 '405': {'chrisnr1998'},
 '406': {'chrisnr1998'},
 '407': {'chrisnr1998'},
 '408': {'chrisnr1998'},
 '409': {'chrisnr1998'},
 '410': {'chrisnr1998'},
 '411': {'chrisnr1998'},
 '412': {'chrisnr1998'},
 '415': {'chrisnr1998'},
 '416': {'chrisnr1998'},
 '417': {'chrisnr1998'},
 '418': {'chrisnr1998'},
 '419': {'chrisnr1998'},
 '420': {'chrisnr1998'},
 '423': {'chrisnr1998'},
 '424': {'chrisnr1998'},
 '425': {'chrisnr1998'},
 '429': {'chrisnr1998'},
 '430': {'chrisnr1998'},
 '431': {'chrisnr1998'},
 '432': {'chrisnr1998'},
 '433': {'chrisnr1998'},
 '434': {'chrisnr1998'},
 '435': {'christopher_brown1991'},
 '436': {'christopher_brown1991'},
 '437': {'christopher_brown1991'},
 '438': {'christopher_brown1991'},
 '440': {'christopher_brown1991'},
 '442': {'christopher_brown1991'},
 '443': {'christopher_brown1991'},
 '444': {'christopher_brown1991'},
 '445': {'christopher_brown1991'},
 '446': {'christopher_brown1991'},
 '448': {'christopher_brown1991'},
 '449': {'christopher_brown1991'},
 '450': {'christopher_brown1991'},
 '451': {'christopher_brown1991'},
 '453': {'christopher_brown1991'},
 '454': {'christopher_brown1991'},
 '456': {'christopher_brown1991'},
 '457': {'christopher_brown1991'},
 '458': {'christopher_brown1991'},
 '459': {'christopher_brown1991'},
 '460': {'christopher_brown1991'},
 '461': {'christopher_brown1991'},
 '462': {'christopher_brown1991'},
 '463': {'christopher_brown1991'},
 '467': {'christopher_brown1991'},
 '468': {'christopher_brown1991'},
 '469': {'christopher_brown1991'},
 '472': {'christopher_brown1991'},
 '473': {'christopher_brown1991'},
 '474': {'georgiamike1968'},
 '475': {'georgiamike1968'},
 '476': {'georgiamike1968'},
 '477': {'georgiamike1968'},
 '478': {'georgiamike1968'},
 '479': {'georgiamike1968'},
 '481': {'georgiamike1968'},
 '482': {'georgiamike1968'},
 '483': {'georgiamike1968'},
 '484': {'georgiamike1968'},
 '485': {'georgiamike1968'},
 '486': {'georgiamike1968'},
 '487': {'georgiamike1968'},
 '488': {'georgiamike1968'},
 '489': {'georgiamike1968'},
 '490': {'georgiamike1968'},
 '491': {'georgiamike1968'},
 '492': {'georgiamike1968'},
 '493': {'georgiamike1968'},
 '494': {'georgiamike1968'},
 '495': {'georgiamike1968'},
 '496': {'georgiamike1968'},
 '497': {'georgiamike1968'},
 '498': {'georgiamike1968'},
 '499': {'georgiamike1968'},
 '500': {'georgiamike1968'},
 '501': {'georgiamike1968'},
 '502': {'georgiamike1968'},
 '503': {'georgiamike1968'},
 '504': {'georgiamike1968'},
 '505': {'georgiamike1968'},
 '506': {'georgiamike1968'},
 '507': {'georgiamike1968'},
 '508': {'georgiamike1968'},
 '509': {'georgiamike1968'},
 '511': {'georgiamike1968'},
 '512': {'georgiamike1968'},
 '513': {'georgiamike1968'},
 '514': {'georgiamike1968'},
 '515': {'georgiamike1968'},
 '516': {'georgiamike1968'},
 '517': {'georgiamike1968'},
 '518': {'georgiamike1968'},
 '519': {'georgiamike1968'},
 '520': {'georgiamike1968'},
 '522': {'georgiamike1968'},
 '523': {'georgiamike1968'},
 '524': {'georgiamike1968'},
 '525': {'georgiamike1968'},
 '526': {'georgiamike1968'},
 '528': {'georgiamike1968'},
 '529': {'georgiamike1968'},
 '530': {'georgiamike1968'},
 '531': {'georgiamike1968'},
 '533': {'georgiamike1968'},
 '534': {'georgiamike1968'},
 '535': {'georgiamike1968'},
 '536': {'georgiamike1968'},
 '537': {'georgiamike1968'},
 '538': {'georgiamike1968'},
 '539': {'georgiamike1968'},
 '542': {'georgiamike1968'},
 '543': {'georgiamike1968'},
 '544': {'georgiamike1968'},
 '545': {'georgiamike1968'},
 '546': {'georgiamike1968'},
 '547': {'georgiamike1968'},
 '548': {'georgiamike1968'},
 '549': {'georgiamike1968'},
 '550': {'georgiamike1968'},
 '551': {'georgiamike1968'},
 '552': {'georgiamike1968'},
 '553': {'georgiamike1968'},
 '554': {'georgiamike1968'},
 '555': {'georgiamike1968'},
 '556': {'georgiamike1968'},
 '557': {'georgiamike1968'},
 '558': {'georgiamike1968'},
 '559': {'georgiamike1968'},
 '560': {'georgiamike1968'},
 '561': {'georgiamike1968'},
 '562': {'georgiamike1968'},
 '563': {'georgiamike1968'},
 '564': {'georgiamike1968'},
 '565': {'georgiamike1968'},
 '566': {'georgiamike1968'},
 '567': {'georgiamike1968'},
 '568': {'georgiamike1968'},
 '569': {'georgiamike1968'},
 '570': {'georgiamike1968'},
 '571': {'georgiamike1968'},
 '572': {'georgiamike1968'},
 '573': {'georgiamike1968'},
 '574': {'georgiamike1968'},
 '575': {'georgiamike1968'},
 '576': {'georgiamike1968'},
 '577': {'georgiamike1968'},
 '579': {'georgiamike1968'},
 '580': {'georgiamike1968'},
 '581': {'georgiamike1968'},
 '582': {'georgiamike1968'},
 '583': {'georgiamike1968'},
 '584': {'georgiamike1968'},
 '585': {'georgiamike1968'},
 '586': {'georgiamike1968'},
 '587': {'georgiamike1968'},
 '588': {'georgiamike1968'},
 '589': {'georgiamike1968'},
 '590': {'georgiamike1968'},
 '591': {'georgiamike1968'},
 '592': {'innocentz6197@sbcglobal.net'},
 '593': {'innocentz6197@sbcglobal.net'},
 '594': {'innocentz6197@sbcglobal.net'},
 '595': {'innocentz6197@sbcglobal.net'},
 '596': {'innocentz6197@sbcglobal.net'},
 '597': {'innocentz6197@sbcglobal.net'},
 '598': {'innocentz6197@sbcglobal.net'},
 '599': {'innocentz6197@sbcglobal.net'},
 '600': {'innocentz6197@sbcglobal.net'},
 '601': {'innocentz6197@sbcglobal.net'},
 '602': {'innocentz6197@sbcglobal.net'},
 '603': {'innocentz6197@sbcglobal.net'},
 '604': {'innocentz6197@sbcglobal.net'},
 '605': {'innocentz6197@sbcglobal.net'},
 '606': {'innocentz6197@sbcglobal.net'},
 '607': {'innocentz6197@sbcglobal.net'},
 '608': {'innocentz6197@sbcglobal.net'},
 '609': {'innocentz6197@sbcglobal.net'},
 '610': {'innocentz6197@sbcglobal.net'},
 '611': {'innocentz6197@sbcglobal.net'},
 '612': {'innocentz6197@sbcglobal.net'},
 '613': {'innocentz6197@sbcglobal.net'},
 '614': {'innocentz6197@sbcglobal.net'},
 '615': {'innocentz6197@sbcglobal.net'},
 '617': {'innocentz6197@sbcglobal.net'},
 '618': {'innocentz6197@sbcglobal.net'},
 '619': {'jackjohnsons7'},
 '620': {'jackjohnsons7'},
 '621': {'jackjohnsons7'}}
author_dict

{'0': {'Billy Joe'},
 '1': {'Billy Joe'},
 '2': {'Billy Joe'},
 '3': {'Billy Joe'},
 '4': {'Billy Joe'},
 '5': {'Billy Joe'},
 '6': {'Billy Joe'},
 '7': {'Billy Joe'},
 '8': {'Carlos Cabrera'},
 '9': {'Carlos Cabrera'},
 '10': {'Carlos Cabrera'},
 '11': {'Carlos Cabrera'},
 '12': {'Carlos Cabrera'},
 '13': {'Carlos Cabrera'},
 '14': {'Carlos Cabrera'},
 '15': {'Carlos Cabrera'},
 '16': {'Carlos Cabrera'},
 '17': {'Carlos Cabrera'},
 '18': {'Carlos Cabrera'},
 '19': {'DB'},
 '20': {'DB'},
 '21': {'DB'},
 '22': {'DB'},
 '23': {'DB'},
 '24': {'DB'},
 '25': {'DB'},
 '26': {'DB'},
 '27': {'DB'},
 '28': {'DB'},
 '29': {'DB'},
 '30': {'DB'},
 '31': {'DB'},
 '32': {'DB'},
 '33': {'DB'},
 '34': {'DB'},
 '35': {'DB'},
 '36': {'DB'},
 '37': {'DB'},
 '38': {'DB'},
 '39': {'DB'},
 '40': {'DB'},
 '41': {'DB'},
 '42': {'DB'},
 '43': {'DB'},
 '44': {'DB'},
 '45': {'DB'},
 '46': {'DB'},
 '47': {'DB'},
 '48': {'DB'},
 '49': {'DB'},
 '50': {'DB'},
 '51': {'DB'},
 '52': {'DB'},
 '53': {'DB'},
 '54': {'DB'

In [12]:
grooming['author'] = grooming['message_idx'].map(author_dict)
grooming['author'] = grooming['author'].apply(lambda x: list(x)[0])
grooming

,source,label,message_idx,author,text,chat_num
0,PJ chats,1,0,Billy Joe,"안녕, 나 매즈야",113
1,PJ chats,1,0,Billy Joe,"안녕, 자기야",113
2,PJ chats,1,0,Billy Joe,거의 일 다 끝났어. 정말 기뻐,113
3,PJ chats,1,0,Billy Joe,안녕 :) 그거 좋은 거지?,113
4,PJ chats,1,0,Billy Joe,응,113
...,...,...,...,...,...,...
7569,PJ chats,1,621,jackjohnsons7,"나 곧 가야 해, 엄마가 소리치고 있어.",31
7570,PJ chats,1,621,jackjohnsons7,그거 좋았어?,31
7571,PJ chats,1,621,jackjohnsons7,그래서 너 하트포드 어디 거리에서 살아?,31
7572,PJ chats,1,621,jackjohnsons7,너한테 안 알려줘.,31


In [13]:
# 같은 author를 가진 데이터를 하나의 row로 합치기
grooming = grooming.groupby('author')['text'].apply(lambda x: '\t'.join(x)).reset_index()
grooming

,author,text
0,Billy Joe,"안녕, 나 매즈야\t안녕, 자기야\t거의 일 다 끝났어. 정말 기뻐\t안녕 :) 그..."
1,Carlos Cabrera,알겠어 안녕\t안녕 이거 카를로스야?\t응 나야\t섹시해\t^_^\t뭐해 베이비\t...
2,DB,안녕 DB!:)\tㅋㅋ 어떻게 지내\t난 좋아;) 너는 어때?\t음 그렇겠지 ㅋㅋ ...
3,F C,안녕 공주\t네 번호를 폰에 저장하고 있었어.. 근데 이름이 뭐야??\t하하. 응 ...
4,J H,안녕?\t미트미에서 온 남자야 ㅋㅋ\t이름이 뭐야??\t제레미\t집에 혼자 있어? ...
5,Josh Mcclendon,"안녕 미트미의 조쉬야\t그래서, 너는 지금까지 어떤 성적인 경험을 해봤는지 궁금해...."
6,Roger Cassidy,안녕 로저야\t수업 중이야\t17살이라고 했잖아.. 그게 왜 나쁘지?\t아니 나쁘지...
7,Ryan Thompson,이거 라이언이야??\t별거 없어 집에서 쉬고 있어\t놀러 갈래\t뭐 진짜로?\t왜 ...
8,allenriley2011,앨런이야 뭐해\t침대에 있는데 아무것도 안 해 껴안고 싶어\t히히.. 나도. 그래\...
9,bossofct,나야 셰인\t안녕ㅎㅎㅎ\t평소처럼 심심해 ㅋㅋ 너는?\t말로니에서 얼마나 멀어?\t...


In [14]:
# author별로 대화 내용을 리스트로 저장
def split_text(df, x, author):
    return df[df[x] == author]['text'].values[0].split('\t')

billy_joe = split_text(grooming, 'author', 'Billy Joe')
carlos_cabrera = split_text(grooming, 'author', 'Carlos Cabrera')
db = split_text(grooming, 'author', 'DB')
f_c = split_text(grooming, 'author', 'F C')
j_h = split_text(grooming, 'author', 'J H')
josh_mcclendon = split_text(grooming, 'author', 'Josh Mcclendon')
roger_cassidy = split_text(grooming, 'author', 'Roger Cassidy')
ryan_thompson = split_text(grooming, 'author', 'Ryan Thompson')
allenriley2011 = split_text(grooming, 'author', 'allenriley2011')
bossofct = split_text(grooming, 'author', 'bossofct')
chrisnr1998 = split_text(grooming, 'author', 'chrisnr1998')
christopher_brown1991 = split_text(grooming, 'author', 'christopher_brown1991')
georgiamike1968 = split_text(grooming, 'author', 'georgiamike1968')
innocentz6197_sbcglobal_net = split_text(grooming, 'author', 'innocentz6197@sbcglobal.net')
jackjohnsons7 = split_text(grooming, 'author', 'jackjohnsons7')
jackjohnsons7

['안녕, 뭐 하고 있어? 나 제러드야.',
 '제러드??',
 '제러드라는 사람 모르는데.',
 'MeetMe에서 만난 사람이야, 바보야. 너 광장까지 걸어올 수 있어? 네 부모님은 뭐라고 하실 거 같아?',
 '오.. ㅎㅎ.. 밥 딜런?',
 '음, 우리 엄마 지금 여기 계셔.',
 '왜, 뭐 하고 싶어?',
 '네가 결정해 ㅎㅎ 뭐 필요한 거 있어?',
 '내가 뭘 필요로 한다고??',
 '네가 뭘 하고 싶은지 알고 싶어.',
 '우리 네 차에서 노는 거야 아니면 뭐야.. 이해 안 돼, 문제 생기고 싶지 않아.',
 '이런, 나 바빴어. 너 처녀 아니지, 맞지?',
 '내가 갈 만한 곳 알아놨어. 네가 엄마한테 허락받은 거라고 약속만 해줘.',
 '알겠어.. 어디야? 친구 집에서 잔다고 엄마한테 말할 수 있어.',
 '그리고 화이트 맨딩고 워리어가 뭐야?? ㅎㅎㅎ',
 '그냥 한두 시간만이야. 친구네 간다고 엄마한테 말해줘, 알겠지.',
 '알겠어, 우리 뭐 할 건데?',
 '몰라 ㅎㅎ 키스하는 거 어때?',
 '키스해?',
 '너도 화이트 맨딩고 워리어 가질 수 있어 ㅎㅎ',
 '그게 무슨 뜻이야? ㅎㅎ',
 '내가 사랑을 나누는 신이라는 뜻인 것 같아 ㅎㅎ 나 지금 네 쪽으로 가고 있어. 30분쯤 후에 도착할게, 알겠지.',
 '나 아직 나가도 되는지 물어보지도 않았어.',
 '그럼 물어봐, 바보야.',
 '엄마가 안 된다고 하면 어쩌지? 나중에 올 수 있어?',
 '친구네 간다고 하고 한두 시간 후에 돌아올 거라고 해.',
 '아니, 나 도탄에 살잖아, 기억나? 난 그냥 시댁에 뭔가를 두고 오려고 해.',
 '음.. 그래서?? 그리 멀진 않잖아.',
 '너 수영하러 간다고 해, 알겠지.',
 '그러니까 네 말은 내가 지금 못 가면 다시 여기 안 올 거라는 거야?',
 '두 가지 일을 하고 있을 때는 그래.',
 '너 좀 재수 없게 행동하는 것 같아.',
 '그러니까 시간 있을 때 연락 줘.',
 '그냥 와, 오래 걸리지 않을 거야. 그리고 아니, 다

In [15]:
window_size = 40
stride = 1
texts = []
idx = 1

for i in range(0, len(data), stride):
    text = "[CLS] " + " ".join(m + " [SEP]" for m in data['text'][i:i+window_size])
    texts.append(text)
    
print(f'input text: {texts[idx]}')

input text: [CLS] 안녕, 자기야 [SEP] 거의 일 다 끝났어. 정말 기뻐 [SEP] 안녕 :) 그거 좋은 거지? [SEP] 응 [SEP] 그럼 뭐 새로운 일 있어? ㅋㅋ [SEP] 없어 [SEP] 샤워할 때 그거 만졌어? ㅋㅋ [SEP] 아니, 빨리 해야 했어 ㅋㅋ [SEP] 잠깐만 [SEP] 미안, 엄마가 자꾸 나한테 이래라저래라 해 ㅋㅋ [SEP] 그거 진짜 짜증나겠다 [SEP] 응, 짜증 나. 너는 엄마랑 잘 지내? [SEP] 그랬다가..엄마가 좀 참견이 많아 [SEP] 여전히 ㅋㅋ [SEP] 너랑 엄마 둘이야? [SEP] 지금 뭐 하고 있어? [SEP] 나도 마찬가지야 :) [SEP] 그 기분 알아 [SEP] 나는 수업에 갈 신발을 가지러 나갈 준비 중이야. 아마 수업 끝나고 돌아올 거야 [SEP] 엄마가 지금 나한테 시비 걸고 있는데, 돌아가면 전화해도 될까 [SEP] 8시쯤 될 거 같아 [SEP] 아... [SEP] 샤워 끝난 후, 핫한 속옷 입었어? [SEP] 엄마가 방에 있으면 몰래 전화할 수 있을 거 같아 [SEP] 집에선 전화 서비스가 안 돼 [SEP] 아마도 [SEP] 그래? [SEP] 나는 주립 숲 근처에 살아 [SEP] 그럼 낮이 항상 나아? [SEP] 정말 안 좋아 [SEP] 야생 동물 보이냐고 ㅋㅋ [SEP] 내일은 나갈 수 있길 바랄게 [SEP] 응, 야생 동물들이 있어 [SEP] 엄마가 집에 없다는 걸 확인하고, 좀 더 이야기할 시간이 필요해서 며칠 간 시간을 가지려고 해. 괜찮을까? [SEP] 좋아 [SEP] 나도 네 목소리가 듣고 싶어, 그래야 네가 할아버지 같은 사람 아니란 걸 알 수 있잖아 ㅋㅋ [SEP] 화났어? [SEP] 할아버지? [SEP] 나 안 늙었어 [SEP] 지금 1분만이라도 나한테 전화했으면 좋겠어 [SEP]


In [16]:
# Sliding Window
def sliding_window(texts, window_size=30, stride=1):
    windows = []
    for i in range(0, len(texts)-window_size+1, stride):
        text = "[CLS] " + " ".join(m + " [SEP]" for m in texts[i:i+window_size])
        windows.append(text)
    return windows


window_billy_joe = sliding_window(billy_joe)
window_billy_joe

['[CLS] 안녕, 나 매즈야 [SEP] 안녕, 자기야 [SEP] 거의 일 다 끝났어. 정말 기뻐 [SEP] 안녕 :) 그거 좋은 거지? [SEP] 응 [SEP] 그럼 뭐 새로운 일 있어? ㅋㅋ [SEP] 없어 [SEP] 샤워할 때 그거 만졌어? ㅋㅋ [SEP] 아니, 빨리 해야 했어 ㅋㅋ [SEP] 잠깐만 [SEP] 미안, 엄마가 자꾸 나한테 이래라저래라 해 ㅋㅋ [SEP] 그거 진짜 짜증나겠다 [SEP] 응, 짜증 나. 너는 엄마랑 잘 지내? [SEP] 그랬다가..엄마가 좀 참견이 많아 [SEP] 여전히 ㅋㅋ [SEP] 너랑 엄마 둘이야? [SEP] 지금 뭐 하고 있어? [SEP] 나도 마찬가지야 :) [SEP] 그 기분 알아 [SEP] 나는 수업에 갈 신발을 가지러 나갈 준비 중이야. 아마 수업 끝나고 돌아올 거야 [SEP] 엄마가 지금 나한테 시비 걸고 있는데, 돌아가면 전화해도 될까 [SEP] 8시쯤 될 거 같아 [SEP] 아... [SEP] 샤워 끝난 후, 핫한 속옷 입었어? [SEP] 엄마가 방에 있으면 몰래 전화할 수 있을 거 같아 [SEP] 집에선 전화 서비스가 안 돼 [SEP] 아마도 [SEP] 그래? [SEP] 나는 주립 숲 근처에 살아 [SEP] 그럼 낮이 항상 나아? [SEP]',
 '[CLS] 안녕, 자기야 [SEP] 거의 일 다 끝났어. 정말 기뻐 [SEP] 안녕 :) 그거 좋은 거지? [SEP] 응 [SEP] 그럼 뭐 새로운 일 있어? ㅋㅋ [SEP] 없어 [SEP] 샤워할 때 그거 만졌어? ㅋㅋ [SEP] 아니, 빨리 해야 했어 ㅋㅋ [SEP] 잠깐만 [SEP] 미안, 엄마가 자꾸 나한테 이래라저래라 해 ㅋㅋ [SEP] 그거 진짜 짜증나겠다 [SEP] 응, 짜증 나. 너는 엄마랑 잘 지내? [SEP] 그랬다가..엄마가 좀 참견이 많아 [SEP] 여전히 ㅋㅋ [SEP] 너랑 엄마 둘이야? [SEP] 지금 뭐 하고 있어? [SEP] 나도 마찬가지야 :) [SEP] 그 기분 알아 [SEP] 나는 수업에 갈 신발을 가지러 나갈 준비 중

In [17]:
# grooming 데이터셋 window 생성
window_billy_joe = sliding_window(billy_joe)
window_carlos_cabrera = sliding_window(carlos_cabrera)
window_db = sliding_window(db)
window_f_c = sliding_window(f_c)
window_j_h = sliding_window(j_h)
window_josh_mcclendon = sliding_window(josh_mcclendon)
window_roger_cassidy = sliding_window(roger_cassidy)
window_ryan_thompson = sliding_window(ryan_thompson)
window_allenriley2011 = sliding_window(allenriley2011)
window_bossofct = sliding_window(bossofct)
window_chrisnr1998 = sliding_window(chrisnr1998)
window_christopher_brown1991 = sliding_window(christopher_brown1991)
window_georgiamike1968 = sliding_window(georgiamike1968)
window_innocentz6197_sbcglobal_net = sliding_window(innocentz6197_sbcglobal_net)
window_jackjohnsons7 = sliding_window(jackjohnsons7)

final_grooming_df = pd.DataFrame({'text': window_billy_joe + window_carlos_cabrera + window_db + window_f_c + window_j_h + window_josh_mcclendon + window_roger_cassidy + window_ryan_thompson + window_allenriley2011 + window_bossofct + window_chrisnr1998 + window_christopher_brown1991 + window_georgiamike1968 + window_innocentz6197_sbcglobal_net + window_jackjohnsons7, 
                                  'labels': 1})
final_grooming_df

,text,labels
0,"[CLS] 안녕, 나 매즈야 [SEP] 안녕, 자기야 [SEP] 거의 일 다 끝났어...",1
1,"[CLS] 안녕, 자기야 [SEP] 거의 일 다 끝났어. 정말 기뻐 [SEP] 안녕...",1
2,[CLS] 거의 일 다 끝났어. 정말 기뻐 [SEP] 안녕 :) 그거 좋은 거지? ...,1
3,[CLS] 안녕 :) 그거 좋은 거지? [SEP] 응 [SEP] 그럼 뭐 새로운 일...,1
4,[CLS] 응 [SEP] 그럼 뭐 새로운 일 있어? ㅋㅋ [SEP] 없어 [SEP]...,1
...,...,...
7134,[CLS] 응.. ㅋㅋ 우리 아직 여기 있어. [SEP] 내가 참는 중인데 네가 방...,1
7135,[CLS] 내가 참는 중인데 네가 방해하고 있어 ㅋㅋㅋ [SEP] ㅋㅋ 닥쳐. [S...,1
7136,"[CLS] ㅋㅋ 닥쳐. [SEP] 저기, 우리 드디어 돌아왔어. [SEP] 이거 원...",1
7137,"[CLS] 저기, 우리 드디어 돌아왔어. [SEP] 이거 원해 베이비? [SEP] ...",1


In [18]:
# non_grooming 데이터셋 window 생성
window_non_grooming = sliding_window(non_grooming['text'].tolist())
final_non_grooming_df = pd.DataFrame({'text': window_non_grooming, 'labels': 0})
final_non_grooming_df

,text,labels
0,[CLS] 오빠야 저녁 식사는 뭐야? [SEP] 나는 아직 뭐 먹을지 모르겠네 니는...,0
1,[CLS] 나는 아직 뭐 먹을지 모르겠네 니는? [SEP] 난 일단 지금 라면 먹고...,0
2,[CLS] 난 일단 지금 라면 먹고 있는데 저녁 모르겠어 ㅋㅋ [SEP] ㅋㅋ 오 ...,0
3,[CLS] ㅋㅋ 오 지금 라면을 먹음 저녁 먹을 수 있나 [SEP] 모르겠어 ㅋㅋ ...,0
4,[CLS] 모르겠어 ㅋㅋ 이제 일어나서 ㅋㅋ [SEP] 어? 이제 일어났다고? ㅋㅋ...,0
...,...,...
13824,[CLS] 맞아! 남자 배우도 최근에 드라마로 되게 잘된 걸로 아는데 케미가 진짜 ...,0
13825,[CLS] 응응 맞아! 부디 좋은 스토리의 드라마이기를...! 하하 [SEP] 요...,0
13826,[CLS] 요즘 영화관 안 간 지 오래됐네. [SEP] 그냥 집에서 옛날 영화들 또...,0
13827,[CLS] 그냥 집에서 옛날 영화들 또 보고 또 보고 하는 거 같아. [SEP] 안...,0


In [19]:
final_df = pd.concat([final_grooming_df, final_non_grooming_df], ignore_index=True)
final_df

,text,labels
0,"[CLS] 안녕, 나 매즈야 [SEP] 안녕, 자기야 [SEP] 거의 일 다 끝났어...",1
1,"[CLS] 안녕, 자기야 [SEP] 거의 일 다 끝났어. 정말 기뻐 [SEP] 안녕...",1
2,[CLS] 거의 일 다 끝났어. 정말 기뻐 [SEP] 안녕 :) 그거 좋은 거지? ...,1
3,[CLS] 안녕 :) 그거 좋은 거지? [SEP] 응 [SEP] 그럼 뭐 새로운 일...,1
4,[CLS] 응 [SEP] 그럼 뭐 새로운 일 있어? ㅋㅋ [SEP] 없어 [SEP]...,1
...,...,...
20963,[CLS] 맞아! 남자 배우도 최근에 드라마로 되게 잘된 걸로 아는데 케미가 진짜 ...,0
20964,[CLS] 응응 맞아! 부디 좋은 스토리의 드라마이기를...! 하하 [SEP] 요...,0
20965,[CLS] 요즘 영화관 안 간 지 오래됐네. [SEP] 그냥 집에서 옛날 영화들 또...,0
20966,[CLS] 그냥 집에서 옛날 영화들 또 보고 또 보고 하는 거 같아. [SEP] 안...,0


In [20]:
final_df.to_csv('/home/k-cat/users/lth/data/FinalDataset.csv', index=False)

# Model Load

In [2]:
# BERT 모델 불러오기
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
model = BertForSequenceClassification.from_pretrained("skt/kobert-base-v1", num_labels=2)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at skt/kobert-base-v1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Tokenizer

In [2]:
chat_data = pd.read_csv('/home/k-cat/users/lth/data/FinalDataset.csv')
chat_data

,text,labels
0,"[CLS] 안녕, 나 매즈야 [SEP] 안녕, 자기야 [SEP] 거의 일 다 끝났어...",1
1,"[CLS] 안녕, 자기야 [SEP] 거의 일 다 끝났어. 정말 기뻐 [SEP] 안녕...",1
2,[CLS] 거의 일 다 끝났어. 정말 기뻐 [SEP] 안녕 :) 그거 좋은 거지? ...,1
3,[CLS] 안녕 :) 그거 좋은 거지? [SEP] 응 [SEP] 그럼 뭐 새로운 일...,1
4,[CLS] 응 [SEP] 그럼 뭐 새로운 일 있어? ㅋㅋ [SEP] 없어 [SEP]...,1
...,...,...
20963,[CLS] 맞아! 남자 배우도 최근에 드라마로 되게 잘된 걸로 아는데 케미가 진짜 ...,0
20964,[CLS] 응응 맞아! 부디 좋은 스토리의 드라마이기를...! 하하 [SEP] 요...,0
20965,[CLS] 요즘 영화관 안 간 지 오래됐네. [SEP] 그냥 집에서 옛날 영화들 또...,0
20966,[CLS] 그냥 집에서 옛날 영화들 또 보고 또 보고 하는 거 같아. [SEP] 안...,0


In [3]:
chat_data['labels'].value_counts()

labels
0    13829
1     7139
Name: count, dtype: int64

In [6]:
def tokenize_function(data):
    return tokenizer(
        data['text'],
        add_special_tokens=False,   # 이미 [CLS], [SEP] 추가됨
        max_length=512,             # 문장 최대 길이
        truncation=True,            # 문장이 max_length보다 길면 자름
        padding=True                # 문장이 max_length보다 짧으면 padding
    )

# 첫 번째 텍스트에 대해 토큰화 수행
tokenized_output = tokenize_function(chat_data.iloc[0])

# 결과 출력
print(f"input_ids: {tokenized_output['input_ids']}")
print(f"token_type_ids: {tokenized_output['token_type_ids']}")
print(f"attention_mask: {tokenized_output['attention_mask']}")
print(f"decoded tokens: {tokenizer.convert_ids_to_tokens(tokenized_output['input_ids'])}")

input_ids: [2, 3135, 5724, 46, 1370, 1986, 7310, 6844, 3, 3135, 5724, 46, 3890, 6844, 3, 873, 3803, 1562, 1363, 5671, 6855, 54, 4102, 1258, 6471, 3, 3135, 5724, 629, 40, 1185, 5377, 4209, 862, 7318, 258, 3, 3612, 3, 1185, 6043, 2145, 2699, 3803, 3868, 258, 517, 492, 492, 3, 3278, 3, 517, 6546, 7018, 7836, 1844, 1185, 5377, 1931, 7248, 6855, 258, 517, 492, 492, 3, 3097, 46, 2563, 5010, 5017, 6855, 517, 492, 492, 3, 3945, 5595, 6150, 3, 2149, 6812, 46, 3257, 5330, 3886, 5628, 1370, 7835, 3670, 6003, 7199, 6023, 6003, 4998, 517, 492, 492, 3, 1185, 5377, 4368, 4396, 7316, 5655, 5406, 3, 3612, 46, 4396, 7316, 1370, 54, 1457, 5760, 3257, 6022, 3942, 4307, 258, 3, 1197, 5782, 5330, 54, 54, 6872, 6141, 5330, 4196, 4427, 5414, 7096, 1952, 3, 3320, 517, 492, 492, 3, 1457, 6022, 3257, 1779, 7096, 6844, 258, 3, 4299, 2145, 4926, 3868, 258, 3, 1370, 5859, 1921, 6844, 629, 40, 3, 1185, 1282, 3175, 3, 1375, 2890, 6896, 781, 3010, 6295, 7088, 767, 6037, 1372, 4249, 4257, 7096, 6844, 54, 3093, 6141, 28

# Dataset

In [ ]:
# train, test 데이터셋 셔플 및 분리
chat_data_suffled = chat_data.sample(frac=1, random_state=42).reset_index(drop=True)
train = chat_data_suffled[:17000]
test = chat_data_suffled[17000:]
print(train.shape, test.shape)

(17000, 2) (3968, 2)


In [8]:
train['labels'].value_counts()

labels
0    11203
1     5797
Name: count, dtype: int64

In [9]:
test['labels'].value_counts()

labels
0    2626
1    1342
Name: count, dtype: int64

In [10]:
# 검증(Vaildation) 데이터셋 분리
train_data, valid_data = train_test_split(train, test_size=0.1, random_state=42)
print(train_data.shape, valid_data.shape)

(15300, 2) (1700, 2)


In [11]:
valid_data['labels'].value_counts()

labels
0    1138
1     562
Name: count, dtype: int64

In [12]:
# Dataset 생성
train_dataset = Dataset.from_pandas(train_data) # pandas DataFrame -> Hugging Face Dataset 형식으로 변환
valid_dataset = Dataset.from_pandas(valid_data)
test_dataset = Dataset.from_pandas(test)

datasets = DatasetDict({'train': train_dataset, 'valid': valid_dataset, 'test': test_dataset}) # train, valid, test 데이터셋을 묶어서 저장
tokenized_datasets = datasets.map(tokenize_function, batched=True) # train, vaild, test 데이터셋에 tokenize_function 적용

Map: 100%|██████████| 3968/3968 [00:02<00:00, 1489.63 examples/s]


In [13]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['text', 'labels', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 15300
    })
    valid: Dataset({
        features: ['text', 'labels', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1700
    })
    test: Dataset({
        features: ['text', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3968
    })
})

# Train

In [14]:
# 모델 저장 경로 설정
model_save_path = '/home/k-cat/users/lth/model_save'

# 학습 파라미터 설정
training_args = TrainingArguments(
    output_dir=model_save_path,                 # 학습 결과 저장 경로
    report_to='wandb',                          # wandb 사용
    num_train_epochs=15,                        # 학습 epoch 설정
    per_device_train_batch_size=32,             # train batch_size 설정
    per_device_eval_batch_size=32,              # test batch_size 설정
    logging_dir=model_save_path+'/logs',        # 학습 log 저장 경로
    logging_steps=100,                          # 학습 log 기록 단위
    save_total_limit=2,                         # 학습 결과 저장 최대 개수
    evaluation_strategy="epoch",                # 매 epoch마다 평가 실행
    save_strategy="epoch",                      # 매 epoch마다 모델 저장
    load_best_model_at_end=True,                # 가장 성능이 좋은 모델을 마지막에 load
)

# 최적화 알고리즘(optimizer) 설정
optimizer = AdamW(model.parameters(), lr=2e-5)

# 스케줄러(scheduler) 설정
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=len(tokenized_datasets['train']) * training_args.num_train_epochs
)

In [15]:
# 성능 평가 지표 설정(binary classification)
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [16]:
# Trainer 생성
trainer = Trainer(
    model=model, 
    tokenizer=tokenizer,
    optimizers=(optimizer, scheduler),
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['valid'],
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=5)]
)

In [ ]:
# 모델 학습
trainer.train()

# Test

In [ ]:
# 저장된 모델 경로
model_checkpoint = "/home/k-cat/users/lth/model_save/checkpoint-2395"

tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
model = BertForSequenceClassification.from_pretrained(model_checkpoint).to('cuda')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


In [18]:
# 테스트(Test) 데이터셋 평가
trainer.evaluate(tokenized_datasets['test'])

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: oiehhun (oiehhun-yonsei-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


{'eval_loss': 1.1668230399664026e-05,
 'eval_model_preparation_time': 0.0015,
 'eval_accuracy': 1.0,
 'eval_f1': 1.0,
 'eval_precision': 1.0,
 'eval_recall': 1.0,
 'eval_runtime': 44.5102,
 'eval_samples_per_second': 89.148,
 'eval_steps_per_second': 2.786}

In [38]:
# 실제 대화 테스트
def predict(chat):
    model.eval()
    tokenized_sent = tokenizer(
        chat,
        add_special_tokens=False,   # 이미 [CLS], [SEP] 추가됨
        max_length=512,             # 문장 최대 길이
        truncation=True,            # 문장이 max_length보다 길면 자름
        padding=True,               # 문장이 max_length보다 짧으면 padding
        return_tensors='pt'         # PyTorch tensor로 반환
    )
    tokenized_sent.to('cuda')
    
    with torch.no_grad():
        outputs = model(
            input_ids=tokenized_sent["input_ids"],
            attention_mask=tokenized_sent["attention_mask"],
            token_type_ids=tokenized_sent["token_type_ids"]
            )
        
    logits = outputs[0]
    logits = logits.detach().cpu()
    result = logits.argmax(-1)  
    
    if result == 0:
        return '일상 대화 😇'
    elif result == 1:
        return '그루밍 대화 👿'

for idx in range(1, 20):
    chat_text = test_dataset[idx]['text']
    label = test_dataset[idx]['labels']
    print(f' Chat: {chat_text}\n label: {label}\n pred: {predict(chat_text)}\n')

 Chat: [CLS] 추억 새록새록~ [SEP] 우주선에서 내려와 토끼가 하는 말 [SEP] 당근 당근 [SEP] 그리고 나 친구들이랑 런닝맨 게임도 하기로 했어! [SEP] 런닝맨 게임은 어떻게 하는 거야? [SEP] 그 포스트잇에 단어 적고 내 이마에 붙이고 하는 게임 알아? [SEP] 웅 알아 [SEP] 그렇게 하는 거야? [SEP] 어 그래서 친구가 포스트잇 하나 챙겨 오기로 했어! [SEP] 아 진짜 자세히 알려 줘 바 바 [SEP] ㅋㅋ 그거 이제 내 이마 위에 있는 단어 맞추는 거잖아! [SEP] 스무고개같이! [SEP] 아 재밌겠다 [SEP] 나도 친구들이랑 해 봐야지 [SEP] ㅋㅋ 어 내가 해 보고 진짜 재밌으면 알려 줄게! [SEP] 웅 고마워 누나 [SEP] 나도 꼭 알려 줘 [SEP] 여친이랑 제일 재밌게 했었던 여행이 언제였어? [SEP] 나 군산 가서 제일 재미있었어 [SEP] 군산? [SEP] 거기 철길 마을 갔었어? [SEP] 철길 마을도 가고 선유도 갔어 [SEP] 선유도 거기는 섬 아니야? [SEP] 군산에서 하룻밤 자고 선유도 갔어 [SEP] ㅋㅋ 홀 대박 거기는 배 타고 들어가야 하는 곳이지? [SEP] 길 뚫어 놔서 차 타고 들어가 [SEP] 홀 대박 ㅋㅋ [SEP] 그러면 들어가고 나오는 시간이 정해져 있는 거네? [SEP] 그런 거 없을걸? [SEP] 근데 바닷길이 열리면 [SEP]
 label: 0
 pred: 일상 대화 😇

 Chat: [CLS] 섹스 하는 거 [SEP] 응 네가 하고 싶다면? [SEP] 왜 웃어 [SEP] ㅋㅋ 네가 바보처럼 굴어서 [SEP] 아니. 그냥 확실히 하려고 ㅋㅋ. [SEP] 근데 알겠어 내가 다 알아볼게 자기야 [SEP] 그래서 할머니 보러 갔구나 [SEP] 좋네. 할머니는 손녀가 이렇게 변태인 줄 모르고 있겠지 [SEP] 오늘 밤 할머니 집에 있어야 할 것 같아 :( [SEP] 그게 싫어? [SEP] 응 너무 지루해 [SEP] 아우우. 나랑 있고 싶지 않아 [SEP] 응! ㅋㅋ 

In [55]:
test = '[CLS] 너랑 하고 싶어. [SEP] 뭘?ㅋㅋ [SEP] 영화 말이야ㅋㅋ [SEP] 나도. [SEP] 내일 영화보고 밥 먹을까? [SEP] 좋아! [SEP]'
print(predict(test))
test = '[CLS] 너랑 하고 싶어. [SEP] 뭘?ㅋㅋ [SEP] 영화 말이야ㅋㅋ [SEP] 나도. [SEP] 내일 영화보고 밥 먹을까? [SEP] 좋아! [SEP] 근데 나 어제 모의고사 망헀어.. [SEP] 나도.. [SEP] 영화 보고나서 같이 공부하자! [SEP] 좋아! [SEP]'
print(predict(test))

그루밍 대화 👿
일상 대화 😇


In [58]:
test = '[CLS] 뽀뽀하고 싶어. [SEP] 뭐랰ㅋㅋㅋ [SEP] 왜 안돼? [SEP] 당연히 안되지~ [SEP] 성인되고 나면~ [SEP] 약속했다~! [SEP]'
print(predict(test))
test = '[CLS] 뽀뽀하고 싶어. [SEP] 뭐랰ㅋㅋㅋ [SEP] 왜 안돼? [SEP] 당연히 안되지~ [SEP] 성인되고 나면~ [SEP] 약속했다~! [SEP] 그래^^ [SEP] 근데 나 어제 모의고사 망헀어.. [SEP] 나도.. [SEP] 내일 같이 공부하자! [SEP] 좋아! [SEP]'
print(predict(test))

그루밍 대화 👿
일상 대화 😇
